In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import getpass

In [ ]:
openai_api_key = getpass.getpass(prompt='Openai Api Key')

## 1. Zero-Shot ReAct Agent (ZERO_SHOT_REACT_DESCRIPTION)

In [12]:
# Define a simple tool
def echo_tool(query):
    return f"You asked: {query}"

dummy_tool = Tool(
    name="EchoTool",
    func=echo_tool,
    description="Repeats whatever input is given."
)
# Initialize OpenAI model
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)

# Create the Zero-Shot ReAct Agent (no tools, just reasoning)
agent = initialize_agent(
    tools=[dummy_tool],  
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent
response = agent.run("What are the three laws of motion?")
print(response)


/var/folders/cf/1n82cp8j5477wf_t828mwrlh0000gn/T/ipykernel_1557/836188319.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/var/folders/cf/1n82cp8j5477wf_t828mwrlh0000gn/T/ipykernel_1557/836188319.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("Wha



> Entering new AgentExecutor chain...
The three laws of motion are fundamental principles formulated by Sir Isaac Newton. I can provide these directly without using any tools.

Final Answer: The three laws of motion, formulated by Sir Isaac Newton, are:

1. **First Law (Law of Inertia):** An object at rest stays at rest, and an object in motion stays in motion with the same speed and in the same direction unless acted upon by an unbalanced external force.

2. **Second Law (Law of Acceleration):** The acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass. This is often expressed with the formula \( F = ma \), where \( F \) is the net force applied to the object, \( m \) is the mass of the object, and \( a \) is the acceleration produced.

3. **Third Law (Action and Reaction):** For every action, there is an equal and opposite reaction. This means that forces always occur in pairs; if object A exerts a force on object B,

## 2. OpenAI Functions Agent (OPENAI_FUNCTIONS)

In [13]:
# Define a simple tool
def get_current_time(*args):
    for arg in args:
        print("argument ",arg)
    from datetime import datetime
    return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

time_tool = Tool(
    name="GetCurrentTime",
    func=get_current_time,
    description="Returns the current time."
)

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# Create an OpenAI Functions Agent with the tool
agent = initialize_agent(
    tools=[time_tool],
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# Run the agent
response = agent.run("What time is it?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `GetCurrentTime` with `now`


argument  now
The current time is 2025-02-08 22:48:17The current time is 22:48 on February 8, 2025.

> Finished chain.
The current time is 22:48 on February 8, 2025.


## 3. ReAct Agent (STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION)

In [14]:
# Define a tool
from langchain.tools import StructuredTool
from pydantic import BaseModel

# Define input schema
class WikiInput(BaseModel):
    query: str

# Define Wikipedia search function
def search_wikipedia(query: str):
    return f"Simulated Wikipedia search results for: {query}"

# Convert function to a StructuredTool
wiki_tool = StructuredTool.from_function(
    func=search_wikipedia,
    name="WikipediaSearch",
    description="Searches Wikipedia for a given query.",
    args_schema=WikiInput,  # ✅ Enforces structured input
)

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# Create a structured ReAct agent
agent = initialize_agent(
    tools=[wiki_tool],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent
response = agent.run("Tell me about quantum mechanics using Wikipedia.")
print(response)



> Entering new AgentExecutor chain...
Thought: I need to search Wikipedia for information on quantum mechanics.
Action:
```
{
  "action": "WikipediaSearch",
  "action_input": {
    "query": "Quantum mechanics"
  }
}
```
Observation: Simulated Wikipedia search results for: Quantum mechanics
Thought:The assistant should review the information provided by the Wikipedia search results on quantum mechanics. Based on the information, the assistant should then formulate a concise and informative response. Since the observation is not provided in this scenario, I'll assume the assistant has obtained a general overview of quantum mechanics from Wikipedia. 

Thought: Quantum mechanics is a fundamental theory in physics that provides a description of the physical properties of nature at the scale of atoms and subatomic particles. It is the foundation of all quantum physics including quantum chemistry, quantum field theory, quantum technology, and quantum information science.

Action:
```
{
  "a

## 4. Conversational Agent (CONVERSATIONAL_REACT_DESCRIPTION)

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# Create a conversational agent
agent = initialize_agent(
    tools=[],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# Run multiple queries to see memory in action
response1 = agent.run("Who was Albert Einstein?")
print(response1)

response2 = agent.run("What was his biggest contribution to physics?")
print(response2)



> Entering new AgentExecutor chain...


/var/folders/cf/1n82cp8j5477wf_t828mwrlh0000gn/T/ipykernel_1557/2918377461.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


Thought: Do I need to use a tool? No
AI: Albert Einstein was a renowned theoretical physicist who is best known for his theory of relativity, one of the two pillars of modern physics (the other being quantum mechanics). He was born in Germany in 1879 and later became a Swiss citizen. His work revolutionized our understanding of the basic laws of the universe.

Einstein's most famous equation, E=mc^2, demonstrates the equivalence of energy (E) and mass (m), with "c" being the speed of light in a vacuum. This equation has profound implications for the understanding of energy and matter.

In 1921, Einstein was awarded the Nobel Prize in Physics for his explanation of the photoelectric effect, which demonstrated the particle-like properties of light. This work was a foundational part of the development of quantum mechanics.

Einstein was also known for his passionate commitment to pacifism, civil rights, education, and Zionism. He passed away in 1955 in Princeton, New Jersey, USA.

> Finis

## 5. Self-Querying Agent (SELF_ASK_WITH_SEARCH)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# ✅ Simulated Search Function (Replace with API calls if needed)
def google_search(query: str):
    search_results = {
        "latest breakthroughs in quantum computing": 
            "IBM's 127-qubit Eagle processor, Google's quantum supremacy, and new quantum algorithms.",
        "top AI companies in 2024": 
            "OpenAI, Google DeepMind, Anthropic, and Meta AI are leading AI research.",
        "climate change solutions": 
            "Renewable energy, carbon capture, and afforestation are key solutions."
    }
    return search_results.get(query.lower(), "No relevant search results found.")

# ✅ Define the Tool (Must be named "Intermediate Answer")
search_tool = Tool(
    name="Intermediate Answer",  # REQUIRED for SELF_ASK_WITH_SEARCH
    func=google_search,
    description="Searches the web for the most relevant and recent information."
)

# ✅ Initialize OpenAI LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key,system_message="Always provide detailed responses in full sentences. Never respond with just 'Yes' or 'No'.")

# ✅ Create the Self-Querying Agent
agent = initialize_agent(
    tools=[search_tool],  # Tool MUST be named "Intermediate Answer"
    llm=llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
    handle_parsing_errors=True
)

# ✅ Run the agent with a complex query
query = "What are the latest breakthroughs in quantum computing?"
response = agent.run(query)

# ✅ Print the response
print("\n🧠 AI Response:", response)


## 6. Plan-and-Execute Agent (PLAN_AND_EXECUTE)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor
from langchain_experimental.plan_and_execute.planners import chat_planner
from langchain_experimental.plan_and_execute.executors import agent_executor
# ✅ Simulated search function
def google_search(query: str):
    search_results = {
        "latest breakthroughs in AI": "OpenAI's GPT-5 research, DeepMind's AlphaFold updates, and advances in multimodal AI.",
        "top space discoveries in 2024": "NASA's James Webb discoveries, exoplanets, and new black hole theories.",
    }
    return search_results.get(query.lower(), "No relevant search results found. Try checking NASA or OpenAI websites.")

# ✅ Define a search tool
search_tool = Tool(
    name="GoogleSearch",
    func=google_search,
    description="Search for the latest scientific and technological advancements."
)

# ✅ Initialize OpenAI LLM
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)

# ✅ Correct Planner & Executor setup for LangChain 0.3.18
planner = chat_planner(llm=llm)  # Replaces `ChatPlanner`
executor = agent_executor(llm=llm, tools=[search_tool])  # Replaces `AgentExecutor`

# ✅ Create Plan-and-Execute Agent
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

# ✅ Run the agent with a complex query
query = "Find the latest breakthroughs in AI and summarize them."
response = agent.run(query)

# ✅ Print the response
print("\n🧠 AI Response:", response)